# Compare 3 Dynamic Programming algorithms for RL:
1. Policy Evaluation 
2. Policy Iteration
3. Value Iteration

(applied to GridWorld)

In [16]:
import numpy as np
from collections import defaultdict

In [11]:
"""
GridWorld 
global variables
"""
n = 5
grid = np.zeros((n, n))

A = [(0, 1), (4, 1), 10]
B = [(0, 3), (2, 3), 5]

actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Policy Evaluation 

This is a generic algorithm to compute the state values for a given policy.

It is formulated for stochastic policies, where the agent decides what action to take stochastically by sampling from a probability distribution over all possible actions a any given state. The distribution varies for each state.
In a degenerate case, the destribution takes value 1 for one action and 0 for all the others (I call this a deterministic policy).

The formula computes the weighted average over all possible actions for each state applying the recursive formula with a discount factor gamma.

In [12]:
def policy_evaluation(policy, gamma, error, grid):
    for c in range(1000):
        grid_new = np.zeros((n, n))
        # (i, j) covers all possible states (aka s')
        # note that in this problem, the result of taking an action is deterministic, 
        # so there is not need to compute an expectation over all the possible outcomes of taking an action
        # as in (3.12)
        for i in range(n):
            for j in range(n):
                st = (i, j)
                pi = policy[st] # get the prob distrib for state st
                # in the case of the special points A and B, the agent chooses an action randomly
                # but the result is always moving to A' and B' regardless of the action
                # so there is no need to compute a weighted sum over all possible actions as in 
                # the other states that are not A or B
                if st == A[0]:
                    st =  A[1]
                    r = A[2]
                    grid_new[i, j] = 1 * (r + gamma * grid[st[0], st[1]])
                elif st == B[0]:
                    st = B[1]
                    r = B[2]
                    grid_new[i, j] = 1 * (r + gamma * grid[st[0], st[1]])
                else:
                    # sum of rewards(*) over all possible actions from current state (i, j)
                    # (*) weighted by the probability of taking each action under policy pi
                    for aIndex, a in enumerate(actions):
                        r = 0
                        stplus1 = (st[0] + a[0], st[1] + a[1])  # element-wise addition of tuples
                        if stplus1[0]<0 or stplus1[0]>n-1 or stplus1[1]<0 or stplus1[1]>n-1:
                            stplus1 = st
                            r = -1
                        grid_new[i, j] += pi[aIndex] * (r + gamma * grid[stplus1[0], stplus1[1]])

        if np.max(np.abs(grid - grid_new)) < error: break
        grid = grid_new

    return grid


pi = [.25, .25, .25, .25]
policy = {(i,j): pi for j in range(n) for i in range(n)}
gamma=0.9 # discount rate   
error=10^-3
state_values = np.zeros((n, n))
state_values = policy_evaluation(policy, gamma, error, state_values)
print np.round(state_values, decimals=2)

[[ 3.31  8.79  4.43  5.32  1.49]
 [ 1.52  2.99  2.25  1.91  0.55]
 [ 0.05  0.74  0.67  0.36 -0.4 ]
 [-0.97 -0.44 -0.35 -0.59 -1.18]
 [-1.86 -1.35 -1.23 -1.42 -1.98]]


# Policy Improvement
The process of making a new policy that improves on an original policy, by making it greedy with respect to the value function of the original policy, is called policy improvement.


This algorithm performs the same recursion as policy evaluation but the policy is improved, instead of assuming a random policy, we apply a greedy choice at each state.

We return an actual policy, that associates an action to each state. It actually associates a probability distribution ove actions to each state, but in this case is a degenerate distribution.


In [13]:
def policy_improvement(state_values, gamma, policy):
    policy_stable = True
    # (4.3):
    # (i, j) covers all possible states (state s at time t)
    for i in range(n):
        for j in range(n):
            st = (i, j)
            # in the case of the special points A and B, the agent chooses an action as per policy
            # but the result is always moving to A' and B' deterministically regardless of the action
            # so the optimal policy used by the agent to choose the action doesn't matter in A and B
            if st == A[0]:
                st = A[1]
                r = A[2]
            elif st == B[0]:
                st = B[1]
                r = B[2]
            else:
                # maximize for all actions 
                action_values = []
                for a in actions:
                    # find state t+1 (given s and a)
                    stplus1 = (st[0] + a[0], st[1] + a[1])  # element-wise addition of tuples
                    
                    # find the reward for transitioning to state t+1
                    r = 0
                    if stplus1[0]<0 or stplus1[0]>n-1 or stplus1[1]<0 or stplus1[1]>n-1:
                        stplus1 = st
                        r = -1
                    # compute the action value with the recursive formula (the algorithm is not recursive thanks to DP)
                    action_values.append(1 * (r + gamma * state_values[stplus1[0], stplus1[1]]))
                    
                # update the policy (associate prob distribution to current state)
                pi = [.0 for _ in actions]
                pi[np.argmax(action_values)] = 1.
                
                if pi != policy[st]:
                    policy_stable = False
                    policy[st] = pi
                    
    return policy, policy_stable

# Policy Iteration

Iterate in a cycle of policy evaluations over a series of policy improvements:
1. Evaluate the initial policy
2. Change the policy with policy improvement
3. Go back to step


Note that in this algorithm we never store the new policy explicitly, the policy is implicit in the state values caluculated with the policy.

In [14]:
def policy_iteration(gamma, error):
    state_values = np.zeros((n, n))
    pi = [.25, .25, .25, .25]
    policy = {(i,j): pi for j in range(n) for i in range(n)}
    
    for i in range(1000):
        state_values = policy_evaluation(policy, gamma, error, state_values)
        
        policy, policy_stable = \
        policy_improvement(state_values, gamma, policy)
        
        print
        print i
        print np.round(state_values, decimals=2)
        
        if policy_stable:
            break
            
    return state_values, policy

gamma=0.9 # discount rate  
error=10^-3
state_values, policy = policy_iteration(gamma, error)


0
[[ 3.31  8.79  4.43  5.32  1.49]
 [ 1.52  2.99  2.25  1.91  0.55]
 [ 0.05  0.74  0.67  0.36 -0.4 ]
 [-0.97 -0.44 -0.35 -0.59 -1.18]
 [-1.86 -1.35 -1.23 -1.42 -1.98]]

1
[[ 21.98  24.42  21.98  18.45  16.61]
 [ 19.78  21.98  19.78  16.61  14.94]
 [ 17.8   19.78  17.8   14.94  13.45]
 [ 16.02  17.8   16.02  13.45  12.11]
 [ 14.42  16.02  14.42  12.11  10.89]]

2
[[ 21.98  24.42  21.98  19.42  17.48]
 [ 19.78  21.98  19.78  17.8   15.73]
 [ 17.8   19.78  17.8   16.02  14.16]
 [ 16.02  17.8   16.02  14.42  12.74]
 [ 14.42  16.02  14.42  12.98  11.47]]

3
[[ 21.98  24.42  21.98  19.42  17.48]
 [ 19.78  21.98  19.78  17.8   16.02]
 [ 17.8   19.78  17.8   16.02  14.42]
 [ 16.02  17.8   16.02  14.42  12.98]
 [ 14.42  16.02  14.42  12.98  11.68]]

4
[[ 21.98  24.42  21.98  19.42  17.48]
 [ 19.78  21.98  19.78  17.8   16.02]
 [ 17.8   19.78  17.8   16.02  14.42]
 [ 16.02  17.8   16.02  14.42  12.98]
 [ 14.42  16.02  14.42  12.98  11.68]]


In [18]:
print "Note that for points A and B the action doesn't matter because all actions lead to A' and B'."
print "In those points the policy is the original uniform distribution, in this case argmax pick the first: actions[0]"
for i in range(n):
    for j in range(n):
        print "(%d,%d): " % (i, j), actions[np.argmax(policy[(i, j)])]

 Note that for points A and B the action doesn't matter because all actions lead to A' and B'.
In those points the policy is the original uniform distribution, in this case argmax pick the first: actions[0]
(0,0):  (0, 1)
(0,1):  (-1, 0)
(0,2):  (0, -1)
(0,3):  (-1, 0)
(0,4):  (0, -1)
(1,0):  (-1, 0)
(1,1):  (-1, 0)
(1,2):  (-1, 0)
(1,3):  (0, -1)
(1,4):  (0, -1)
(2,0):  (-1, 0)
(2,1):  (-1, 0)
(2,2):  (-1, 0)
(2,3):  (-1, 0)
(2,4):  (-1, 0)
(3,0):  (-1, 0)
(3,1):  (-1, 0)
(3,2):  (-1, 0)
(3,3):  (-1, 0)
(3,4):  (-1, 0)
(4,0):  (-1, 0)
(4,1):  (-1, 0)
(4,2):  (-1, 0)
(4,3):  (-1, 0)
(4,4):  (-1, 0)


# Value Iteration

For an example of value iteration see GridWorld in chapter 3